In [11]:

import sys
lib_decomren = "/home/dotamthuc-3090/Projects/ViewSynthesis/DeConRen"
sys.path.insert(0, lib_decomren)

import itertools
import numpy as np
import torch
import decompress_render

In [13]:
np.array(
        tuple(itertools.product([-1, 0, 1], [-1, 0, 1], [-1, 0, 1]))) + 1

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2]])

In [43]:
pos_np = np.array(tuple(itertools.product([0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3])))
pos = torch.tensor(pos_np, dtype=torch.int64).cuda()
nlevel = torch.tensor([2], dtype=torch.int8).cuda()
morton_code = decompress_render.utils.position_to_mortoncode(pos, nlevel).cpu().numpy()
combi = np.concatenate([pos_np, morton_code], axis=1)

combi[np.argsort(morton_code[:, 0])]

array([[ 0,  0,  0,  0],
       [ 0,  0,  1,  1],
       [ 0,  1,  0,  2],
       [ 0,  1,  1,  3],
       [ 1,  0,  0,  4],
       [ 1,  0,  1,  5],
       [ 1,  1,  0,  6],
       [ 1,  1,  1,  7],
       [ 0,  0,  2,  8],
       [ 0,  0,  3,  9],
       [ 0,  1,  2, 10],
       [ 0,  1,  3, 11],
       [ 1,  0,  2, 12],
       [ 1,  0,  3, 13],
       [ 1,  1,  2, 14],
       [ 1,  1,  3, 15],
       [ 0,  2,  0, 16],
       [ 0,  2,  1, 17],
       [ 0,  3,  0, 18],
       [ 0,  3,  1, 19],
       [ 1,  2,  0, 20],
       [ 1,  2,  1, 21],
       [ 1,  3,  0, 22],
       [ 1,  3,  1, 23],
       [ 0,  2,  2, 24],
       [ 0,  2,  3, 25],
       [ 0,  3,  2, 26],
       [ 0,  3,  3, 27],
       [ 1,  2,  2, 28],
       [ 1,  2,  3, 29],
       [ 1,  3,  2, 30],
       [ 1,  3,  3, 31],
       [ 2,  0,  0, 32],
       [ 2,  0,  1, 33],
       [ 2,  1,  0, 34],
       [ 2,  1,  1, 35],
       [ 3,  0,  0, 36],
       [ 3,  0,  1, 37],
       [ 3,  1,  0, 38],
       [ 3,  1,  1, 39],


In [8]:

morton_code = torch.zeros((10, 1), dtype=torch.int64).cuda()
nlevel = torch.ones((1), dtype=torch.int8).cuda()
decompress_render.utils.mortoncode_to_position(morton_code, nlevel)

tensor([[1, 0, 0]], device='cuda:0')

In [ ]:
# # SH colors
# SH_C0 = 0.28209479177387814
# SH_C1 = 0.4886025119029199
# SH_C2 = [ 1.0925, -1.0925, 0.3153, -1.0925, 0.5462 ]
# SH_C3= [ -0.5900, 2.8906, -0.4570, 0.3731, -0.4570, 1.4453, -0.5900]

# # 
# def computeColorFromSH( vox_center, cam_center, sh0, sh, degree=3):
#     # vox_center, cam_center: array of 3D position - [x, y, z]
#     # sh0: [cR, cG, cB]
#     # sh: array (num_SH_coefs-1) x 3
    
#     # Normalized ray direction
#     dir = vox_center - cam_center
#     dir = dir * 1 / (dir[0]**2, dir[1]**2, dir[2]**2)

#     # Result is vector 3: [R, G, B]
#     result = SH_C0 * sh0

#     if (degree > 0):
#         x = dir[0]
#         y = dir[1]
#         z = dir[2]
#         result = result - SH_C1 * y * sh[0] + SH_C1 * z * sh[1] - SH_C1 * x * sh[2]

#         if (degree > 1):
#             xx = x * x
#             yy = y * y
#             zz = z * z
#             xy = x * y
#             yz = y * z
#             xz = x * z

#             result = result + \
#                 SH_C2[0] * xy * sh[3] + \
#                 SH_C2[1] * yz * sh[4] + \
#                 SH_C2[2] * (2.0 * zz - xx - yy) * sh[5] + \
#                 SH_C2[3] * xz * sh[6] + \
#                 SH_C2[4] * (xx - yy) * sh[7]

#             if (degree > 2):
#                 result = result + \
#                     SH_C3[0] * y * (3.0 * xx - yy) * sh[8] + \
#                     SH_C3[1] * xy * z * sh[9] + \
#                     SH_C3[2] * y * (4.0 * zz - xx - yy) * sh[10] + \
#                     SH_C3[3] * z * (2.0 * zz - 3.0 * xx - 3.0 * yy) * sh[11] + \
#                     SH_C3[4] * x * (4.0 * zz - xx - yy) * sh[12] + \
#                     SH_C3[5] * z * (xx - yy) * sh[13] + \
#                     SH_C3[6] * x * (xx - 3.0 * yy) * sh[14]
        
#     result = result + 0.5

#     # RGB colors are clamped to non-negative values.
#     result[0] *= (result[0] > 0.0)
#     result[1] *= (result[1] > 0.0)
#     result[2] *= (result[2] > 0.0)
#     return result
